In [1]:
import os
from pathlib import Path

In [2]:
from sandbox.agents.DDPG import DDPG
from sandbox.utils.ddpg_utils import to_tensor
from sandbox.utils.common_utils import load_config, set_logging
from sandbox.env.robosuite_lift import GymWrapper

In [3]:
import robosuite as suite
import gym
import glfw
import torch

In [4]:
def train(agent, env, num_epochs, num_episodes, episode_horizon, warmup, render):
    """
    This function trains a ddpg agent given training parameters provided by config file.
    """
    log.info("Setting up Agent and Environment.")
    obs = env.reset()
    iteration = 0
    log.info("Creating weights folders.")
    if not os.path.exists("weights"):
        os.mkdir("weights")

    log.info("Environment: {} \n Agent: {}\n".format(config["environment"]["env_name"], agent.name))


    log.info("Commencing Warmup")
    while iteration <= warmup:
        obs = env.reset()
        agent.s_t = obs
        done = False
        while done==False:
            action = agent.random_action()
            obs, reward, done, info = env.step(action)
            agent.observe(reward, obs, done)
            iteration+=1

    log.info("Commencing Training")
    for epoch in range(num_epochs):
        epoch_reward = 0
        for episode in range(num_episodes):
            obs = env.reset()
            agent.s_t = obs
            done=False
            steps = 0
            while (done==False) & (steps<=episode_horizon): 
                action = agent.select_action(obs)     
                obs, reward, done, info = env.step(action)
                steps += 1
                epoch_reward+=reward
                agent.observe(reward, obs, done)
                agent.update_parameters()
        log.info("reward for epoch_{}: {}".format(epoch,epoch_reward/num_episodes))
    
        model = "./weights/model_{}.pt".format(epoch+1)
        torch.save(agent.actor.state_dict(), model)

        model_c = "./weights/model_actor_target_{}.pt".format(epoch+1)
        torch.save(agent.actor_target.state_dict(), model_c)

        model_c = "./weights/model_critic_{}.pt".format(epoch+1)
        torch.save(agent.critic.state_dict(), model_c)

        model_c = "./weights/model_critic_target_{}.pt".format(epoch+1)
        torch.save(agent.critic_target.state_dict(), model_c)

    
        if config["environment"]["env_source"]=="openai_gym":		
            env_to_wrap = gym.make(config["environment"]["env_name"])
        else:
            env_to_wrap = env

In [5]:
def render_rollout(env, epoch, play):
    """
    This function renders a rollout given a certain set of model weights.
    """
    if config["environment"]["env_source"]=="openai_gym":		
        env_to_wrap = gym.make(config["environment"]["env_name"])
    else:
        env_to_wrap = env

    env = env_to_wrap


    agent = DDPG(**config["ddpg_agent"], load=True, epoch=epoch)

    for i in range(play):
        obs = env.reset()
        done = False
        steps = 0
        while (done==False) & (steps <= config["training_params"]["episode_horizon"]):
            env.render()
            action = agent.actor(to_tensor(obs).to(agent.actor.device)).to('cpu').detach().numpy()
            obs, reward, done, info = env.step(action)
    glfw.terminate()

In [6]:
# Getting path variables
sandbox_path = str(Path(os.getcwd()).parent)

logging_path = sandbox_path + "/Project_files/sandbox/config/default_logger.conf"
config_path = sandbox_path + "/Project_files/sandbox/config/lift_ddpg.yaml"
# Set logging
log = set_logging(logging_path)

# Read config
config = load_config(config_path)
# Train cartpole
if config["environment"]["env_source"]=="openai_gym":
    env = gym.make(config["environment"]["env_name"])
elif config["environment"]["env_source"]=="stanford_robosuite":
    env = suite.make(**config["robosuite_env"])
    env = GymWrapper(env, keys=config["robosuite_keys"])
else:
    log.info("Error setting the enviroment.")
    
env.close()

Creating window glfw


/home/nikhil/anaconda3/envs/torchenv/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


# Training

In [7]:
# create an agent
# agent = DDPG(**config["ddpg_agent"])

# Train
# train(agent, env, **config["training_params"])

# Continue training by loading

(NOT Recommended!!)

In [8]:
# epoch_to_load = 15
# agent = DDPG(**config["ddpg_agent"], load=True, epoch=epoch_to_load)
# train(agent, env, **config["training_params"]) 

# Validating

In [9]:
epoch_to_load = 180
games = 3

render_rollout(env, epoch_to_load, games)

Loading Agent and critic....
Loaded the networks..
